In [1]:
from sklearn.datasets import make_classification

# Generate a 2-class classification dataset with 5 features and 200 instances
X, y = make_classification(n_samples=200, n_features=5, n_informative=2, n_classes=2, random_state=42)

X_, y_ = make_classification(n_samples=50, n_features=5, n_informative=2, n_classes=2, random_state=42)


In [4]:
from AITIA.heuristics import DisjunctSize, DisjunctClassPercentage

In [3]:
test = DisjunctSize()
test.fit(X,y)
test.decision_tree

{'depth': 0,
 'decision': {'feature_name': 3, 'threshold': -0.15020010247826576},
 'left': {'depth': 1,
  'decision': {'feature_name': 1, 'threshold': -0.4398489147424698},
  'left': {'depth': 2,
   'decision': {'feature_name': 3, 'threshold': -0.380782812833786},
   'left': {'depth': 3,
    'decision': {'feature_name': 0, 'threshold': 0.9023169875144958},
    'left': {'depth': 4,
     'decision': {'feature_name': 3, 'threshold': -0.68701171875},
     'left': {'depth': 5, 'instances_count': 1, 'instances_by_class': {0: 1}},
     'right': {'depth': 5,
      'instances_count': 1,
      'instances_by_class': {1: 1}}},
    'right': {'depth': 4, 'instances_count': 5, 'instances_by_class': {1: 5}}},
   'right': {'depth': 3, 'instances_count': 4, 'instances_by_class': {0: 4}}},
  'right': {'depth': 2,
   'decision': {'feature_name': 3, 'threshold': -0.7986275255680084},
   'left': {'depth': 3, 'instances_count': 58, 'instances_by_class': {0: 58}},
   'right': {'depth': 3,
    'decision': {'fe

In [5]:
test = DisjunctClassPercentage()
test.fit(X,y,max_depth=2,balanced=True)
test.decision_tree

{'depth': 0,
 'decision': {'feature_name': 3, 'threshold': -0.15020010247826576},
 'left': {'depth': 1,
  'decision': {'feature_name': 0, 'threshold': 0.8179221451282501},
  'left': {'depth': 2,
   'instances_count': 80,
   'instances_by_class': {0: 78, 1: 2}},
  'right': {'depth': 2,
   'instances_count': 11,
   'instances_by_class': {0: 5, 1: 6}}},
 'right': {'depth': 1,
  'decision': {'feature_name': 1, 'threshold': 0.6275590360164642},
  'left': {'depth': 2,
   'instances_count': 106,
   'instances_by_class': {1: 93, 0: 13}},
  'right': {'depth': 2, 'instances_count': 3, 'instances_by_class': {0: 3}}}}